In [ ]:
!pip3 install snscrape

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 KB 3.2 MB/s eta 0:00:00


In [ ]:
!nvidia-smi

Mon Feb 20 12:33:15 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P0    23W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import pandas as pd
from tqdm.notebook import tqdm
import snscrape.modules.twitter as sntwitter

In [ ]:
scraper=sntwitter.TwitterSearchScraper("#Kyiv")
tweets=[]

for i,tweet in enumerate(scraper.get_items()):
  data=[
      tweet.date,
      tweet.id,
      tweet.rawContent,
      tweet.user.username,
      tweet.likeCount,
      tweet.retweetCount
  ]
  tweets.append(data)
  if i > 200:
    break

In [ ]:
tweet_df=pd.DataFrame(tweets,columns=["date","id","content","username","likecount","retweet"])

In [ ]:
tweet_df

,date,id,content,username,likecount,retweet
0,2023-02-20 12:33:13+00:00,1627647562814480384,The Bear can bite us. The Dragon is spitting f...,gavinstener,0,0
1,2023-02-20 12:33:09+00:00,1627647542996369409,⚡️ #Donbass 🇷🇺 #Ukraine ☠️ @BFMTV @LCI et leur...,Circonscripti18,0,0
2,2023-02-20 12:33:05+00:00,1627647527552970757,The best part of this week is that it's not ov...,movenpickng,1,0
3,2023-02-20 12:32:58+00:00,1627647500097097729,"🇺🇸Joe: Wolodymyr, I want icecream🍦 \n\n🇺🇦Wolod...",BrennpunktUA,0,0
4,2023-02-20 12:32:58+00:00,1627647496620064774,Biden announced the promise of Western countri...,IdeallyaNews,0,0
...,...,...,...,...,...,...
197,2023-02-20 12:07:08+00:00,1627640996438724608,Everyone reached Kyiv except for the little 🇷🇺...,iNtRoVeRt_Ed,25,10
198,2023-02-20 12:07:04+00:00,1627640979368087552,How to find a partner for English speaking pra...,MdAbdul33336064,0,0
199,2023-02-20 12:07:02+00:00,1627640972191404034,"""Tu chodzi o coś więcej niż tylko Ukrainę. Cho...",RubyTuesday828,1,0
200,2023-02-20 12:07:01+00:00,1627640965899907077,Prezydent USA Joe Biden i @ZelenskyyUa przed ...,RubyTuesday828,1,0


In [ ]:
import pandas as pd
import numpy as np
import scipy as sp

import nltk
import re

from wordcloud import WordCloud

from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

# Downloading all the requirements
!pip install scikit-multilearn
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')

!pip install contractions
import contractions

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline, make_union

from tqdm import tqdm

# !pip install -U imbalanced-learn

import matplotlib.pyplot as plt
from matplotlib import rcParams
import seaborn as sns

# Setting global parameters for the Plots
rcParams['figure.figsize'] = 18, 9 
rcParams['axes.spines.top'] = False
rcParams['axes.spines.right'] = False
rcParams['lines.linewidth'] = 2.5
rcParams['xtick.labelsize'] = 'smaller'
rcParams['ytick.labelsize'] = 'smaller'
rcParams['axes.labelpad'] = 15

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
class preprocess:

    def __init__(self):
        pass

    def clean(self, df, column):
      
        # applying all the methods in sequence
        df[column] = df[column].str.lower()
        df[column] = self.fix_contractions(df, column)

        # df[column] = df[column].apply(self.rm_decontracted)
        df[column] = df[column].apply(self.rm_urls)
        df[column] = df[column].apply(self.rm_html_tags)
        df[column] = df[column].apply(self.rm_non_alpha)
        df[column] = df[column].apply(self.rm_hash)
        # df[column] = self.rm_stopwords(df, column)

        # df[column] = df[column].apply(self.rm_patterns_removal) #ZZZZZZ REMOVER
        df[column] = df[column].apply(self.rm_repeated_text) ### ZZZZ TO Z
        df[column] = df[column].apply(self.rm_numword_removal)
        df[column] = self.rm_whitespace(df, column)

        # lowercasing the text


        df = self.lemmatization(df, column)
        # df[column] = self.rm_stopwords(df, column)
        # df = self.remove_less_2(df, column)
    
        return df

    def rm_hash(self,text):
        return re.sub(r'#',' ', text)

    def rm_urls(self, text):
        return re.sub(r'http\S+', ' ', text)
    
    def rm_html_tags(self, text):
        clean = re.compile('<.*?>')
        return re.sub(clean, ' ', text)

    def rm_non_alpha(self, text):
        return re.sub('[^a-zA-Z]', ' ', text)

    def rm_whitespace(self, df, column):
        return df[column].apply(lambda x : " ".join(x.split()))

    def lemmatization(self, df, column):
        lemma = WordNetLemmatizer()

        # Lemmatizing with respect to the verb form and noun form
        df[column] = df[column].apply(lambda x : " ".join([lemma.lemmatize(word = w, pos='v') for w in x.split(" ")]))
        df[column] = df[column].apply(lambda x : " ".join([lemma.lemmatize(word = w, pos='n') for w in x.split(" ")]))

        # df[column] = df[column].apply(lambda x : " ".join([lemma.lemmatize(word = w, pos='a') for w in x.split(" ")]))	
        # df[column] = df[column].apply(lambda x : " ".join([lemma.lemmatize(word = w, pos='r') for w in x.split(" ")]))	
        # df[column] = df[column].apply(lambda x : " ".join([lemma.lemmatize(word = w, pos='s') for w in x.split(" ")]))		

        return df

    def rm_stopwords(self, df, column):

        # Generating all the stop words of english and removing no,not nor as they are significantly important
        stop = stopwords.words('english')
        not_stop_words = {'no', 'not', 'nor','very','but'}
        modified_stop = [ele for ele in stop if ele not in not_stop_words]

        return df[column].apply(lambda x : ' '.join([word for word in x.split(' ') if word not in modified_stop]))

    # As the words with less than length 2 will not contribute to toxicity in a general manner
    def _clean_small(self, text):
        return re.sub(r'\b\w{1,2}\b', '', text)

    # After every re.sub we get varied amount of whitespaces so it is essential to remove white spaces
    def remove_less_2(self, df, column):
        df[column] = df[column].apply(self._clean_small)
        df[column] = self.rm_whitespace(df, column)
        return df

    # zzzzzzzzzz remover
    def rm_patterns_removal(self,text): 
        return re.sub("\\s*\\b(?=\\w*(\\w)\\1{2,})\\w*\\b",' ',text)

    #function to remove words with numbers
    def rm_numword_removal(self,text):
        return re.sub("\S*\d\S*", " ", text).strip()

    def rm_decontracted(self,text):
      phrase = re.sub(r"\'t", " not", text)
      phrase = re.sub(r"\'re", " are", phrase)
      phrase = re.sub(r"\'s", " is", phrase)
      phrase = re.sub(r"\'d", " would", phrase)
      phrase = re.sub(r"\'ll", " will", phrase)
      phrase = re.sub(r"\'ve", " have", phrase)
      phrase = re.sub(r"\'m", " am", phrase)
      return phrase

    # zzzzzzzzzz updater to z remover
    def rm_repeated_text(self,text):
      return re.sub(r'(.)\1{2,}', r'\1', text) 

    def fix_contractions(self, df, column):
        return df[column].apply(contractions.fix)

In [ ]:
!pip install langdetect

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
df_test = preprocess().clean(tweet_df, 'content')
df_test

,date,id,content,username,likecount,retweet
0,2023-02-20 12:33:13+00:00,1627647562814480384,the bear can bite u the dragon be spit fire a ...,gavinstener,0,0
1,2023-02-20 12:33:09+00:00,1627647542996369409,donbas ukraine bfmtv lci et leurs alli s colla...,Circonscripti18,0,0
2,2023-02-20 12:33:05+00:00,1627647527552970757,the best part of this week be that it be not o...,movenpickng,1,0
3,2023-02-20 12:32:58+00:00,1627647500097097729,joe wolodymyr i want icecream wolodymyr o k jo...,BrennpunktUA,0,0
4,2023-02-20 12:32:58+00:00,1627647496620064774,biden announce the promise of western country ...,IdeallyaNews,0,0
...,...,...,...,...,...,...
197,2023-02-20 12:07:08+00:00,1627640996438724608,everyone reach kyiv except for the little bunk...,iNtRoVeRt_Ed,25,10
198,2023-02-20 12:07:04+00:00,1627640979368087552,how to find a partner for english speak practi...,MdAbdul33336064,0,0
199,2023-02-20 12:07:02+00:00,1627640972191404034,tu chodzi o co wi cej ni tylko ukrain chodzi o...,RubyTuesday828,1,0
200,2023-02-20 12:07:01+00:00,1627640965899907077,prezydent usa joe biden i zelenskyyua przed wy...,RubyTuesday828,1,0


In [ ]:
# df=df_test['content']
# df.to_csv('file1.csv')

In [ ]:
df_test['content']

'algu m que use ou ja tenha usado o app wise aquele de transferir dinheiro pro exterior kyiv biden'

In [ ]:
from langdetect import detect
lst=""
file=[]
j=0
for i,x in enumerate(df_test['content']):
  if(detect(x)=='en'):
      file.append(x)

In [ ]:
file_df=pd.DataFrame(file,columns=["content"])

# Spacy

In [ ]:
import spacy
from spacy.tokens.doc import Doc

In [ ]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")
nlp.get_pipe("ner").labels

('CARDINAL',
 'DATE',
 'EVENT',
 'FAC',
 'GPE',
 'LANGUAGE',
 'LAW',
 'LOC',
 'MONEY',
 'NORP',
 'ORDINAL',
 'ORG',
 'PERCENT',
 'PERSON',
 'PRODUCT',
 'QUANTITY',
 'TIME',
 'WORK_OF_ART')

In [ ]:
# text=spacy_tweets[6]
# doc = nlp(text)
  
# # Token and Tag
# for token in doc:
#   print(token, token.pos_)
  
# # # You want list of Verb tokens
# # print("Verbs:", [token.text for token in doc if token.pos_ == "VERB"])

soviet ADJ
storm NOUN
documentary NOUN
all DET
episode NOUN
from ADP
to ADP
history NOUN
of ADP
russia PROPN
war PROPN
film NOUN
starmediaen PROPN
kyiv PROPN
germany PROPN
ukraine PROPN
usa PROPN
bachmut PROPN


In [ ]:
spacy_tweets = []

for doc in nlp.pipe(file_df.content, n_process = -1):
    spacy_tweets.append(doc)

In [ ]:
spacy_tweets[6]

soviet storm documentary all episode from to history of russia war film starmediaen kyiv germany ukraine usa bachmut

In [ ]:
useful_entities = [u'VERB', u'NORP', u'FACILITY', u'ORG', u'GPE', u'LOC', u'EVENT', u'DATE', u'TIME']
other = [u'NUM', u'NOUN', u'PROPN']

In [ ]:
content_tweets = []
for single_tweet in tqdm(spacy_tweets):
    single_tweet_content = []
    for token in single_tweet: 
        if ((token.ent_type_ in useful_entities)  
            or (token.pos_ == u'NUM')):
            single_tweet_content.append(token)
    content_tweets.append(single_tweet_content)

100%|██████████| 110/110 [00:00<00:00, 54388.00it/s]


In [ ]:
tweet_num = 3
print ("original_tweet \n" + str(spacy_tweets[tweet_num]) 
       + "\n\noriginal_tweet\n" + str([str(x) for x in spacy_tweets[tweet_num]])
       + "\n\ncontent_tweet\n" + str(content_tweets[tweet_num])
      )

original_tweet 
biden announce the promise of western country to transfer about tank to kyiv country promise ukraine kyiv

original_tweet
['biden', 'announce', 'the', 'promise', 'of', 'western', 'country', 'to', 'transfer', 'about', 'tank', 'to', 'kyiv', 'country', 'promise', 'ukraine', 'kyiv']

content_tweet
[]


In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
!pip install textacy

In [ ]:
from textacy.representations import Vectorizer
import textacy.representations

In [ ]:
vectorizer = Vectorizer(tf_type="linear", idf_type="smooth")

In [ ]:
term_matrix = vectorizer.fit_transform([tok.lemma_ for tok in doc] for doc in spacy_tweets)

In [ ]:
np_matrix = term_matrix.todense()

In [ ]:
np_matrix.shape

(110, 843)

In [ ]:
for key in sorted(vectorizer.vocabulary_terms):
    print (key, vectorizer.vocabulary_terms[key])

In [ ]:
for token in content_tweets[8]:
    print (token.lemma_, vectorizer.vocabulary_terms[token.lemma_], np.max(np_matrix[:,vectorizer.vocabulary_terms[token.lemma_]]))

ukraine 629 4.2026653232422575


In [ ]:
tfidf_dict = {}
content_vocab = []
for tweet in content_tweets: 
    for token in tweet: 
        if token.lemma_ not in tfidf_dict: 
            content_vocab.append(token.lemma_)
            tfidf_dict[token.lemma_] = np.max(np_matrix[:,vectorizer.vocabulary_terms[token.lemma_]])

In [ ]:
for key in sorted(tfidf_dict):
    print ("WORD:" + str(key) + " -- tf-idf SCORE:" +  str(tfidf_dict[key]))

In [ ]:
# !pip install pymprog
from pymprog import *

In [ ]:
begin('COWTS')
x = var('x', len(spacy_tweets), bool)
x[8]

0 <= x[8] <= 1 binary

In [ ]:
y = var('y', len(content_vocab), bool)
len(y)

92

In [ ]:
content_tweets

In [ ]:
maximize(sum(x) + sum([tfidf_dict[content_vocab[j]]*y[j] for j in range(len(y))]));

In [ ]:
## Maximum length of the entire tweet summary

# Was 150 for the tweet summary, 
# But generated a 1000 word summary for CONABS
L = 1000

# hiding the output of this line since its a very long sum 
sum([x[i]*len(spacy_tweets[i]) for i in range(len(x))]) <= L;

In [ ]:
def content_words(i):
    '''Given a tweet index i (for x[i]), this method will return the indices of the words in the 
    content_vocab[] array
    Note: these indices are the same as for the y variable
    '''
    tweet = spacy_tweets[i]
    content_indices = []
    
    for token in tweet:
        if token.lemma_ in content_vocab:
            content_indices.append(content_vocab.index(token.lemma_))
    return content_indices

In [ ]:
def tweets_with_content_words(j):
    '''Given the index j of some content word (for content_vocab[j] or y[j])
    this method will return the indices of all tweets which contain this content word
    '''
    content_word = content_vocab[j]
    
    index_in_term_matrix = vectorizer.vocabulary_terms[content_word]
    
    matrix_column = np_matrix[:, index_in_term_matrix]
    
    return np.nonzero(matrix_column)[0]

In [ ]:
for j in range(len(y)):
    sum([x[i] for i in tweets_with_content_words(j)])>= y[j]

In [ ]:
for i in range(len(x)):
    sum(y[j] for j in content_words(i)) >= len(content_words(i))*x[i]

In [ ]:
solve()
result_x =  [value.primal for value in x]
result_y = [value.primal for value in y]

In [ ]:
end()
chosen_tweets = np.nonzero(result_x)
chosen_words = np.nonzero(result_y)

In [ ]:
len(chosen_tweets[0]), len(chosen_words[0])

(39, 88)

--------------
the bear can bite u the dragon be spit fire a blinken poke it with a stick in munich this morning i wake to see potus just hr drive time from moscow reagan be hr when in berlin how can any of this end well for america needmoreweapons kyiv
--------------
the best part of this week be that it be not over yet ice cream for breakfast yes please abujatwittercommunity dessert kanoicecream mondayblues kyiv
--------------
soviet storm documentary all episode from to history of russia war film starmediaen kyiv germany ukraine usa bachmut
--------------
kyiv socialjustice sjw equality vina
--------------
potus be offer a meet with zelenskyyua at the border or in lviv but he insist on a visit to kyiv roll stone with reference to a white house official
--------------
current condition in kyiv ukraine
--------------
biden in kyiv the u be back in europe a a security power quite literally that cannot be take for grant and that be why it be so important that we european show respect fo

# NLTK

In [ ]:
import nltk
from nltk.tokenize import TweetTokenizer
from nltk import pos_tag, ne_chunk
from nltk.tree import Tree
from nltk.tag.stanford import StanfordNERTagger


In [ ]:
tokenizer = TweetTokenizer()

In [ ]:
nltk_tweets = []

for tweet in file_df.content:
    nltk_tweets.append(tokenizer.tokenize(tweet))

In [ ]:
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
pos_tag(nltk_tweets[100])

[('biden', 'JJ'),
 ('arrive', 'NN'),
 ('in', 'IN'),
 ('kyiv', 'NN'),
 ('by', 'IN'),
 ('train', 'NN'),
 ('this', 'DT'),
 ('morning', 'NN'),
 ('the', 'DT'),
 ('new', 'JJ'),
 ('york', 'NN'),
 ('time', 'NN'),
 ('ukrainewar', 'JJ'),
 ('standwithukraine', 'JJ'),
 ('ukraina', 'JJ'),
 ('ukraine', 'JJ'),
 ('ukrainianarmy', 'NN'),
 ('ukrainerussianwar', 'NN')]

In [ ]:
nltk_pos = []

for tweet in nltk_tweets:
    nltk_pos.append(pos_tag(tweet))

In [ ]:
nltk_pos

[[('but', 'CC'),
  ('not', 'RB'),
  ('the', 'DT'),
  ('eye', 'NN'),
  ('of', 'IN'),
  ('israeli', 'JJ'),
  ('here', 'RB'),
  ('create', 'VBP'),
  ('an', 'DT'),
  ('elect', 'JJ'),
  ('dictatorship', 'NN'),
  ('be', 'VB'),
  ('more', 'RBR'),
  ('important', 'JJ'),
  ('than', 'IN'),
  ('that', 'DT'),
  ('biden', 'JJ'),
  ('visit', 'NN'),
  ('to', 'TO'),
  ('kyiv', 'VB')],
 [('potus', 'NN'),
  ('be', 'VB'),
  ('spend', 'VBN'),
  ('presidentsday', 'JJ'),
  ('in', 'IN'),
  ('kyiv', 'NN'),
  ('i', 'JJ'),
  ('my', 'PRP$'),
  ('president', 'NN'),
  ('he', 'PRP'),
  ('be', 'VB'),
  ('a', 'DT'),
  ('brave', 'JJ'),
  ('good', 'JJ'),
  ('man', 'NN')],
 [('ukrainian', 'JJ'),
  ('drone', 'NN'),
  ('drop', 'NN'),
  ('compilation', 'NN'),
  ('kyiv', 'VBD'),
  ('germany', 'JJ'),
  ('ukraine', 'JJ'),
  ('usa', 'NN'),
  ('bachmut', 'NN')],
 [('president', 'NN'),
  ('biden', 'NNS'),
  ('have', 'VBP'),
  ('make', 'VBN'),
  ('a', 'DT'),
  ('surprise', 'NN'),
  ('visit', 'NN'),
  ('to', 'TO'),
  ('kyiv', 'VB'

In [ ]:
from langdetect import detect
lst=""
file=[]
j=0
for i,x in enumerate(df_test['content']):
    if(i%25==0):
      file.append(lst)
      lst=""

In [ ]:
lst

'.\n guess who just get window on hardware'

In [ ]:
file

['.\n michael rogtallbloke about the only demographic not represent be the pimply student who vote for it amp will not have to live with it presumably they be all dress up in black and mask to protest against that bunch of racist amp fascist by smash window amp steal from shop',
 '.\n cpu mine monero xmr on chuwi hi x window tablet see the hash rate and the computer spec below easy one click mine software want to give it a try cryptocurrency makemoneyonline crypto minergate cryptoinvestment.\n kabula get drink in popopo island and be absolutely pay for the window.\n fast screen recorder freeware record your online video meet video class self tutorial and more window.\n articulate shareware create online and mobile course that work on every device window.\n teac hr audio player freeware teac hr audio player be a music playback tool for teac usb audio dac device window.\n if you say my eye be beautiful it be because they be look at you for my eye be just the window my feel come through.\

In [ ]:
file_df=pd.DataFrame(file,columns=["content"])

In [ ]:
file_df['content']

0    .\n michael rogtallbloke about the only demogr...
1    .\n cpu mine monero xmr on chuwi hi x window t...
2    .\n do not catch my newsletter on friday about...
3    .\n should i open the window.\n moonlight floo...
4    .\n window voice recorder.\n bpscustoms the le...
5    .\n isimiya sndr sndl opm z window note x mml ...
Name: content, dtype: object

In [ ]:
file_df
file_df.to_csv('file_val.csv')

In [ ]:
df = pd.read_csv('/content/file1.csv')
df


,Unnamed: 0,content,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,summary\n
0,0,.\n prithvishawscufflecase prithvishaw prithvi...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Prithvishaw selfie controversy is a shameful v...
1,1,.\n stay strong prithvi shaw whole india count...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Prithvishaw should be punished for taking a se...
2,2,.\n prithvishaw sapnagill carattack mumbai.\n ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Prithvishaw sapnagill carattack mumbai is an e...
3,3,.\n influencer prithvishaw prithvishawselfie s...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,We support and trust Prithvi Shaw and urge him...
4,4,.\n chapri influencer prithvishaw supportprith...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Prithvishaw selfie row escalates after woman c...
5,5,.\n in sb ko content chaiye chahe road pe nang...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Prithvishaw should put illiterate influencers ...
6,6,.\n and mandira communicate party it involve t...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Prithvishaw sapnagill is an Indian cricketer w...
7,7,.\n i stand with prithvi shaw prithvishaw.\n i...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Prithvi Shaw has been attacked by two female i...
8,8,.\n hustle video of cricketer prithvishaw amp ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Prithvishaw is the next vinod kambli in cricke...


In [ ]:
# !pip install summarizer

In [ ]:
# !pip install bert-extractive-summarizer

In [ ]:
# from summarizer import Summarizer

# body = 'Text body that you want to summarize with BERT'
# body2 = 'Something else you want to summarize with BERT'
# model = Summarizer()
# model(body)
# model(body2)

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


'Something else you want to summarize with BERT'

In [ ]:
from summarizer import Summarizer

body = '''
Air India has signed what can now safely be referred to as the mother of all aviation deals, at least in India.
 
On February 14, the country’s oldest airline, now firmly under the ownership of the Tata group, announced that it is purchasing 470 aircraft from airline manufacturers Airbus and Boeing. The combined value of the deals is estimated at some $80 billion. The deals were announced during interactions between the Tata leadership along with Prime Minister Narendra Modi and French President Emmanuel Macron and US President Joe Biden separately.
 
The US expects that the sale will support over one million American jobs across 44 states while France reckons that the contract is a milestone in the friendly relations between the countries.
 
The order comprises 40 Airbus A350s, 20 Boeing 787s, and 10 Boeing 777-9s widebody aircraft, as well as 210 Airbus A320/321 Neos and 190 Boeing 737 MAX single-aisle aircraft. The A350 aircraft will be powered by Rolls-Royce engines and the B777/787s by engines from GE Aerospace. All single-aisle aircraft will be powered by engines from FM International.
 
The first of the new aircraft will enter service in late-2023, with the bulk to arrive from mid-2025 onwards. In the meantime, Air India has started taking delivery of 11 leased B777 and 25 A320 aircraft as part of what the airline’s CEO Campbell Wilson calls his plan to restore Air India to its former glory.
 
"Air India is on a large transformation journey across safety, customer service, technology, engineering, network and human resources,” N Chandrasekaran, chairman of Air India and Tata Sons, said in a statement. “Modern, efficient fleet is a fundamental component of this transformation. This order is an important step in realising Air India's ambition, articulated in its Vihaan.Al transformation programme, to offer a world-class proposition serving global travellers with an Indian heart.
 
The Boeing orders, based on the planes’ list prices, are pegged around $45.9 billion, including options for 70 more aircraft, while Airbus’s valuations are unknown. It is, however, likely that the airline has negotiated discounts from both manufacturers. Air India’s deal of 470 aircraft with options for another 70 means that its tally has surpassed a 2019 order by IndiGo for 300 planes, and that by American Airlines for 460 planes in 2011.
 
“The order puts Air India on an extremely strong footing, as it prepares to take on the Gulf and European carriers head-on in the international market,” says Vinamra Longani, head of operations at Sarin & Co, a law firm specialising in aircraft leasing and finances. “Air India is showing signs of improvement in its service standards and operation, and with the new A350s coming in later this year with brand new cabin products, competition will soon intensify on multiple international routes.”

Also read: Air India: Will we see a new avatar of the beleaguered airline in 2023?
 
Of the total order, 400 aircraft are single-aisle aircraft which means that the aircraft would likely be used for domestic or short-haul international operations. Some will also be used for operations at the new low-cost arm comprising Air India Express and AirAsia India. The others are dual-aisle planes that will be used for long-haul operations.
 
“The order clearly indicates a fleet renewal which will help with costs, capabilities and customer experience,” says Satyendra Pandey, managing partner of aviation services firm AT-TV. “It is expected that while the narrowbodies will service the domestic sector, the Airbus-wide bodies are to largely serve the ultra-long-haul segments. This is interesting as currently the entire long-haul and ultra-long haul fleet comprise Boeing aircraft.”
 
Since the takeover by the Tata group, the airline has gone on an offensive under a new management team led by Wilson, a former CEO of Scoot, the low-cost arm of Singapore Airlines. The group has also announced that by 2024, it will merge full-service carrier Vistara to form a combined entity that will operate the country’s largest full-service carrier with a market share of some 20 percent.
 
“These new aircraft will modernise the airline's fleet and onboard product, and dramatically expand its global network,” added Chandrasekharan. “The growth enabled by this order will also provide unparalleled career opportunities for Indian aviation professionals and catalyse accelerated development of the Indian aviation ecosystem."
 
The Tata group is currently looking to build a market share of 30 percent in the domestic aviation space through its three entities, including Air India, Vistara and Air Asia India. “Leaving aside the different airlines and different brands that sit under the Tata portfolio, what I see from an Air India perspective is the need for a full service and low-cost proposition,” Wilson had told Forbes India in October 2022. “So, our ambition is to build a world-class full-service carrier and a world-class low-cost carrier under the Air India Group and operate them synergistically.”
 
“In the domestic market, Air India’s two-pronged strategy of having a full-service carrier and a low-cost airline will ensure it can operate the product most suited to a particular city,” adds Longani. “As markets operated by the low-cost arm mature, they can then be transferred to the full-service arm.”
 
India’s domestic market is currently dominated by IndiGo, which has been expanding at breakneck speed. The airline currently boasts a fleet of over 300 aircraft, the highest in the country. It operates the Airbus A320 CEO and NEO, the A321 NEO, and the ATR 72-600 aircraft. IndiGo has a domestic market share of 56.2 percent, ferrying over 65 lakh passengers a month and operates over 1,600 daily flights, and connects 76 domestic destinations and 26 international ones.
 
In all, the Tata group currently owns four airlines in India with 100 percent stake in Air India, AirAsia India and Air India Express in addition to a 51 percent stake in Vistara. India’s domestic aviation market is currently led by low-cost carriers that control as much as 80 percent of the market. The Tata group’s total fleet size stands at 219, including Air India’s 113 aircraft, AirAsia India’s 28 aircraft, Vistara’s 54, and Air India Express’ 24 jets.
 
“We’ve been clear that it’s a five-year programme,” Wilson had told Forbes India. “The first six months are about addressing the accumulated grievances and issues that have historically been holding the airline back, addressing them at war scale, and then moving on to an 18-month programme of investing in systems, people, aircraft, training, internal products to make a clear statement of intent. Then the subsequent couple of years is the climb phase, which is where we do all those million and one little things that are necessary to go from very good to world-class.”

Air India returned to the Tata group after 70 years when the government chose the Mumbai-headquartered salt-to-steel conglomerate as part of its disinvestment plan.
 
JRD Tata, former chairman of the Tata group, had founded Air India in 1932 and the government took over the airline in 1953 as part of a nationalisation plan. The government purchased a majority stake in the carrier from Tata Sons though its founder JRD Tata continued as chairman till 1977. The company was renamed Air India International Limited, and the domestic services were transferred to Indian Airlines as part of a restructuring.
 
Over the years, the airline has come to symbolise everything wrong with governmental interference in the running of day-to-day operations of a company. Much of that is largely due to an ill-timed move by the Indian government in 2007 when it merged Air India—then flying internationally—with domestic carrier Indian Airlines, creating a bigger entity, the National Aviation Company of India Ltd. Both Air India and Indian Airlines were making losses at that time, bleeding Rs541 crore and Rs240 crore, respectively, in 2007.
 
The merged company had more than 30,000 employees—256 per plane—in 2007, twice the global norm. That meant the company was spending almost one-fifth of its revenue on employee pay and benefits, while other airlines traditionally spent only about one-tenth. Besides, the 2008 global economic collapse threw operational expenses out of gear as oil prices skyrocketed. As more private airlines took to flying on routes that once belonged to Air India, business plummeted. That meant, over time, the airline racked up debt, which stood at Rs60,000 crore by August 2021, and was steadily losing money on operations.
 
Since coming to power in 2014, the Modi government had made multiple attempts to sell the airline to private companies, before managing to complete the process in October last year. As part of its bid, the Tata group’s wholly owned subsidiary Talace Pvt Ltd put an enterprise value (EV) bid at Rs18,000 crore with debt to be retained at Rs15,300 crore and a cash component of Rs2,700 crore. Tata’s bid was higher than the Ajay Singh-led consortia’s EV bid at Rs15,100 crore. The Tatas took 100 percent control of Air India, the low-cost carrier AI Express, and Air India’s 50 percent stake in ground handling firm AI-SATS.
 
Since then, the airline has been trying to improve its poor reputation with improved on-time performances and improved in-flight experiences. The airline has also been offering voluntary retirement scheme for its employees as part of its plan to streamline operations and bring young talent into the organisation. With India emerging as the world’s third-largest aviation market, the Tata group is clear about its intention to tap into the massive opportunity as millions of Indians look to take to skies in the years to come.
 
The latest order is also the first in nearly two decades for Air India after an order for 111 aircraft—68 from Boeing and 43 from Airbus—in 2005 worth $10.8 billion. “The order aims to both modernise and expand the airline’s fleet with the objective of creating a larger and premium full-service carrier that will cater to the growing travel demand in the region,” Airbus said in a statement.
 
However, even as orders are placed, effective utilisation remains key if Air India needs to find itself back to its glory. “In airline planning, there is an adage ‘order the aircraft, spec the aircraft & deploy the aircraft’,” adds Pandey. “It is the deployment that translates into cash flow and eventual profits, so with the order in place now the focus has to be on specifications which include engine selection, maintenance contracts, cabin configurations, layouts, and range specifications, and then deployment of the assets where the right aircraft is deployed for the right routes.”
'''

model = Summarizer()
result = model(lst, num_sentences=2)
full = ''.join(result)

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [ ]:
full

'sapna gill amp her friend ask for some selfies amp be demand more which shaw do not like so she attack the car he be travel in with a baseball bat now she be arrest by the police sai baba prithvishaw sapnagill. prithvi shaw and his friend be attack for deny selfies in mumbai by fan prithvishaw sapnagill indiancricket viralvideos.'

In [ ]:
lst

'.\n sapna gill amp her friend ask for some selfies amp be demand more which shaw do not like so she attack the car he be travel in with a baseball bat now she be arrest by the police sai baba prithvishaw sapnagill.\n after see the prithvi shaw incident indian cricketer to social medium influencers who wish to take selfie with them be prithvishaw prithivishaw sapnagill mumbaipolice.\n sapna gill v prithvi shaw prithvi shaw attack video prithvi shaw fight video prithvi shaw fight sapnagill prithvishaw youtube video.\n if chapri tiktoker amp her friend can frame prithvishaw with crop clip inspite of them attack the poor guy first amp create a narrative against him think what risk any guy or girl without any social influence have in metro sapnagill.\n fir prithvishaw prithvishaw sapnagill mumbaipolice.\n ye kya ho gaya nifty nifty sensex stockmarket madefortrade swarabhasker indian shehzada varundhawan kabzaa sapnagill prithvishaw tiger kartikaaryan allinforchennaiyin wearebfc valentinesd

In [ ]:
body

'\nAir India has signed what can now safely be referred to as the mother of all aviation deals, at least in India.\n \nOn February 14, the country’s oldest airline, now firmly under the ownership of the Tata group, announced that it is purchasing 470 aircraft from airline manufacturers Airbus and Boeing. The combined value of the deals is estimated at some $80 billion. The deals were announced during interactions between the Tata leadership along with Prime Minister Narendra Modi and French President Emmanuel Macron and US President Joe Biden separately.\n \nThe US expects that the sale will support over one million American jobs across 44 states while France reckons that the contract is a milestone in the friendly relations between the countries.\n \nThe order comprises 40 Airbus A350s, 20 Boeing 787s, and 10 Boeing 777-9s widebody aircraft, as well as 210 Airbus A320/321 Neos and 190 Boeing 737 MAX single-aisle aircraft. The A350 aircraft will be powered by Rolls-Royce engines and th

In [ ]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

In [ ]:
# !pip install -U transformers
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 50.5 MB/s eta 0:00:00


In [ ]:
model_name="transformersbook/pegasus-samsum"

In [ ]:
pegasus_tokenizer = PegasusTokenizer.from_pretrained(model_name)
pegaus_model = PegasusForConditionalGeneration.from_pretrained(model_name)

In [ ]:
# lst = dataset_ts[0]["content"]

# reference = dataset_ts[0]['summary']

In [ ]:
tokens = pegasus_tokenizer(lst,truncation=True,padding="longest",return_tensors="pt")

In [ ]:
tokens

{'input_ids': tensor([[  110,   107,  1229, 16355, 13653, 59101,   129,  5386,   115,  2523,
          3758,   110, 36250,   122,   668,  3017,  5386,   115,   718,   326,
          3575,  3033, 28090,  5332,  2523,  5995,   110, 15302, 29862, 14510,
           111, 10182,  2278, 13653, 59101,  1416,   369,   114,   809,   113,
           422,   444, 17032,  9599, 21926,   114,   476,   111,   902,   798,
           431, 39120,   307,  7770, 18000,   107, 39120,   307,  7770, 18000,
         23898,  2558, 30757, 39120,   307,  7770, 18000,   116, 10384,  2558,
         30757,  1152, 27170, 39120,   307,  7770, 18000, 15006, 39120,   307,
          7770, 18000, 63538, 60108,   107, 63538, 27251, 67110, 39120,   307,
          7770, 18000,   170,   462,  2970,  9899,   118, 63538,   129,   938,
          2281,   141,   114,  1590,   111,   215,  1151,   115,   109, 34069,
           110, 75330,   345,   113,   109, 67110,   110,   116,   238, 13438,
         63538, 39120,   307,  7770, 1

In [ ]:
lst

'.\n coleg cambria be locate in north east wale with five campus locate in deeside wrexham northop llysfasi and yale cambria college offer a variety of course include vocational qualification apprenticeship a level and higher education program prithvishaw.\n prithvishaw sapnagill prithvishawsapnagillclash prithvishawvideo prithvishaw mumbaipolice.\n mumbai indian cricketer prithvishaw who play domestic cricket for mumbai be recently attack by a woman and her friend in the santa cruz area of the cricketer s home turf mumbai prithvishaw sapnagill selfie.\n update on prithvi shaw case ali kasif khan lawyer of sapna gill who involve in clash with indian cricketer prithvishaw.\n this be the reason both of them should be take into custody amp conduct fair investigation before lawyer get involve god know what will happen now because sapnagill be a woman influencer she could file fake case amp destroy prithvishaw s career in a snap we have such law to be honest.\n finally we be get the full st

In [ ]:
enoded_summ = pegaus_model.generate(**tokens)

Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "forced_eos_token_id": 1,
  "length_penalty": 0.8,
  "max_length": 128,
  "min_length": 32,
  "num_beams": 8,
  "pad_token_id": 0,
  "transformers_version": "4.26.1"
}

/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 128 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [ ]:
dec = pegasus_tokenizer.decode(enoded_summ[0],skip_special_tokens=True)

In [ ]:
dec

'.<n>i stand with prithvishaw and strongly condemn the aggressive behavior display by sapnagill and her accomplice.<n>time to support prithvishaw people need to stop call influencers to idiot like sapnagill.'

In [ ]:
reference

'Sapna Gill arrested by Mumbai police for allegedly attacking Indian cricketer Prithvishaw with a baseball bat. Shameful viral video of Indian cricketer Prithvi Shaw being attacked by a woman and her friend should be punished severely.'

In [ ]:
dec

'Indian cricketer prithvishaw was attacked by a woman and her friend in the Santacruz area. bhojpuri actress sapnagill was arrested by police for her role in the attack.'

# Pegasus Fine Tuned

In [ ]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

In [ ]:
from transformers import pipeline, set_seed

import matplotlib.pyplot as plt

import pandas as pd
from datasets import load_dataset, load_metric

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

import nltk
from nltk.tokenize import sent_tokenize

from tqdm import tqdm
import torch

nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

device = "cuda" if torch.cuda.is_available() else "cpu"

model_ckpt = "google/pegasus-cnn_dailymail"

tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

In [ ]:
def generate_batch_sized_chunks(list_of_elements, batch_size):
    """split the dataset into smaller batches that we can process simultaneously
    Yield successive batch-sized chunks from list_of_elements."""
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]

def calculate_metric_on_test_ds(dataset, metric, model, tokenizer, 
                               batch_size=9, device=device, 
                               column_text="article", 
                               column_summary="highlights"):
    article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
    target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))

    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)):
        
        inputs = tokenizer(article_batch, max_length=600,  truncation=True, 
                        padding="max_length", return_tensors="pt")
        
        summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                         attention_mask=inputs["attention_mask"].to(device), 
                         length_penalty=0.8, num_beams=8, max_length=50)
        ''' parameter for length penalty ensures that the model does not generate sequences that are too long. '''
        
        # Finally, we decode the generated texts, 
        # replace the <n> token, and add the decoded texts with the references to the metric.
        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True, 
                                clean_up_tokenization_spaces=True) 
               for s in summaries]      
        
        decoded_summaries = [d.replace("<n>", " ") for d in decoded_summaries]
        
        
        metric.add_batch(predictions=decoded_summaries, references=target_batch)
        
    #  Finally compute and return the ROUGE scores.
    score = metric.compute()
    return score

In [ ]:
print("\nContent:")

print(df["content"][0])

print("\nSummary:")

print(df["summary\n"][0])


Content:
.
 prithvishawscufflecase prithvishaw prithvishawselfie prithvishawvideo more detail.
 prithvishaw prithvishawselfie.
 cricket sportsnews prithvishaw sapnagill klrahul indvsaus.
 sportsnews cricketer prithvishaw.
 once you start sleep late there be no go back prithvishaw abdevilliers indvsaus.
 indian cricketer prithvi shaw attack in mumbai for deny selfies prithvishaw mb.
 indian cricketer prithvi shaw attack in mumbai for deny selfies prithvishaw mb.
 shameful viral video on social medium whatever it be but it be not a human it be an animal it be a request to the modi govt that it should be punish severely immediate action prithvishaw sapnagill th test askdk pathaanday askdk.
 prithvi shaw selfie controversy fight video girl prithvishaw prithvishawfightvideo vknews prithvishaw indianteam watch video.
 who be sapnagill the girl see fight with cricketer prithvishaw in viral video werindia.
 i be learn computer trendinghastag viratkohli prithvishaw prithvishawselfie prithvisha

In [ ]:
df_val = pd.read_csv('/content/file_val.csv')
df_val=df_val.drop(['Unnamed: 0'], axis=1)
df_val

,content,summary
0,.\n thing allegedly escalate after the cricket...,Mumbai police have arrested social media influ...
1,.\n cricketer prithvishaw attack by fan for de...,Cricketer prithvishaw attacked by fan for deny...
2,.\n sapna gill file an fir on prithvi shaw see...,Sapna gill and her friend have been arrested f...


In [ ]:
df_test = pd.read_csv('/content/file_test.csv')
df_test=df_test.drop(['Unnamed: 0'], axis=1)
df_test

,content,summary
0,.\n coleg cambria be locate in north east wale...,Sapna Gill arrested by Mumbai police for alleg...
1,.\n prithvishaw sapnagill selfie controversey ...,Prithvishaw was recently attacked by a woman a...


In [ ]:
df_tr = pd.DataFrame(columns=['content','summary'],index=range(0,9))

In [ ]:
for i,x in enumerate(df['content']):
  df_tr['content'][i]=df['content'][i]
  df_tr['summary'][i]=df['summary\n'][i]
df_tr

,content,summary
0,.\n prithvishawscufflecase prithvishaw prithvi...,Prithvishaw selfie controversy is a shameful v...
1,.\n stay strong prithvi shaw whole india count...,Prithvishaw should be punished for taking a se...
2,.\n prithvishaw sapnagill carattack mumbai.\n ...,Prithvishaw sapnagill carattack mumbai is an e...
3,.\n influencer prithvishaw prithvishawselfie s...,We support and trust Prithvi Shaw and urge him...
4,.\n chapri influencer prithvishaw supportprith...,Prithvishaw selfie row escalates after woman c...
5,.\n in sb ko content chaiye chahe road pe nang...,Prithvishaw should put illiterate influencers ...
6,.\n and mandira communicate party it involve t...,Prithvishaw sapnagill is an Indian cricketer w...
7,.\n i stand with prithvi shaw prithvishaw.\n i...,Prithvi Shaw has been attacked by two female i...
8,.\n hustle video of cricketer prithvishaw amp ...,Prithvishaw is the next vinod kambli in cricke...


In [ ]:
df_tr['content'] = df_tr['content'].tolist()
df_tr['summary'] = df_tr['summary'].tolist()
df_tr.dtypes

content    object
summary    object
dtype: object

In [ ]:
pipe = pipeline('summarization', model = model_ckpt )

pipe_out = pipe(df["content"][3])

print(pipe_out)

[{'summary_text': 'influencers sapnagill sapnagill athiaditi and manaifarmhouse .<n>We support and trust you ye ladkiya jinhone apne jindagi m kuchh nhi kiya h inse darne ki jarurat nhi h victim card khelno do inhe you be the future of Indian cricket team prithvishaw.<n>We stand with prithvi shaw until everything become clear medium trial already declare the culprit anti woman and all prithvishaw prithvishawvideo.'}]


In [ ]:
import pyarrow as pa
import pyarrow.dataset as ds
import pandas as pd
from datasets import Dataset

# df = pd.DataFrame({'a': [0,1,2], 'b': [3,4,5]})
dataset = ds.dataset(pa.Table.from_pandas(df_tr).to_batches())

### convert to Huggingface dataset
arr_tr = Dataset(pa.Table.from_pandas(df_tr))

dataset = ds.dataset(pa.Table.from_pandas(df_val).to_batches())

### convert to Huggingface dataset
arr_val = Dataset(pa.Table.from_pandas(df_val))

dataset = ds.dataset(pa.Table.from_pandas(df_test).to_batches())

### convert to Huggingface dataset
arr_test = Dataset(pa.Table.from_pandas(df_test))


In [ ]:
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]

rouge_metric = load_metric('rouge')

score = calculate_metric_on_test_ds(arr_tr, rouge_metric, model_pegasus, tokenizer, column_text = 'content', column_summary='summary')

<ipython-input-18-ec7ec40866cc>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge_metric = load_metric('rouge')
100%|██████████| 1/1 [00:06<00:00,  6.86s/it]


In [ ]:
rouge_dict = dict((rn, score[rn].mid.fmeasure ) for rn in rouge_names )

pd.DataFrame(rouge_dict, index = ['pegasus'])

,rouge1,rouge2,rougeL,rougeLsum
pegasus,0.232739,0.08115,0.166097,0.168317


In [ ]:
arr_test

Dataset({
    features: ['content', 'summary'],
    num_rows: 2
})

In [ ]:
def convert_examples_to_features(example_batch):
    input_encodings = tokenizer(example_batch['content'] , max_length = 1024, truncation = True )
    
    with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer(example_batch['summary'], max_length = 128, truncation = True )
        
    return {
        'input_ids' : input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids']
    }
    
dataset_tr = arr_tr.map(convert_examples_to_features, batched = True)
dataset_val = arr_val.map(convert_examples_to_features, batched = True)
dataset_ts = arr_test.map(convert_examples_to_features, batched = True)

  0%|          | 0/1 [00:00<?, ?ba/s]

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:3581: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
from transformers import DataCollatorForSeq2Seq

seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)

In [ ]:
from transformers import TrainingArguments, Trainer

trainer_args = TrainingArguments(
    output_dir='pegasus-samsum', num_train_epochs=1, warmup_steps=500,
    per_device_train_batch_size=1, per_device_eval_batch_size=1,
    weight_decay=0.01, logging_steps=10,
    evaluation_strategy='steps', eval_steps=500, save_steps=1e6,
    gradient_accumulation_steps=16
) 

In [ ]:
trainer = Trainer(model=model_pegasus, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_tr, 
                  eval_dataset=dataset_val)

In [ ]:
trainer.train()

score = calculate_metric_on_test_ds(
    dataset_ts, rouge_metric, trainer.model, tokenizer, batch_size = 2, column_text = 'content', column_summary= 'summary'
)

rouge_dict = dict((rn, score[rn].mid.fmeasure ) for rn in rouge_names )

pd.DataFrame(rouge_dict, index = [f'pegasus'] )

The following columns in the training set don't have a corresponding argument in `PegasusForConditionalGeneration.forward` and have been ignored: summary, content. If summary, content are not expected by `PegasusForConditionalGeneration.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 9
  Num Epochs = 1
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 16
  Total optimization steps = 1
  Number of trainable parameters = 568699904
You're using a PegasusTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is 

Step,Training Loss,Validation Loss




Training completed. Do not forget to share your model on huggingface.co/models =)


  0%|          | 0/1 [00:00<?, ?it/s]Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "forced_eos_token_id": 1,
  "length_penalty": 0.8,
  "max_length": 128,
  "min_length": 32,
  "num_beams": 8,
  "pad_token_id": 0,
  "transformers_version": "4.26.1"
}

100%|██████████| 1/1 [00:01<00:00,  1.66s/it]


,rouge1,rouge2,rougeL,rougeLsum
pegasus,0.372206,0.179984,0.234724,0.234724


In [ ]:
dataset_tr[0]['summary']

'Prithvishaw selfie controversy is a shameful viral video that should be punished severely. Prithvi Shaw was allegedly manhandled and attacked with a baseball bat outside a hotel in Mumbai, leading to an investigation and arrest.'

In [ ]:
model_pegasus.save_pretrained('peg_model')

Configuration saved in peg_model/config.json
Configuration saved in peg_model/generation_config.json
Model weights saved in peg_model/pytorch_model.bin


In [ ]:
tokenizer.save_pretrained('tokenizer')

tokenizer config file saved in tokenizer/tokenizer_config.json
Special tokens file saved in tokenizer/special_tokens_map.json


('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/spiece.model',
 'tokenizer/added_tokens.json',
 'tokenizer/tokenizer.json')

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("tokenizer")

loading file spiece.model
loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json


In [ ]:
ls

file1.csv      file_val.csv     peg_model/    tokenizer/
file_test.csv  pegasus-samsum/  sample_data/


In [ ]:
gen_kwargs = {"length_penalty": 0.8, "num_beams":8, "max_length": 128}



sample_text = dataset_ts[0]["content"]

reference = dataset_ts[0]['summary']

pipe = pipeline("summarization", model="peg_model",tokenizer=tokenizer)
     


print("\nReference Summary:")
print(reference)


print("\nModel Summary:")
print(pipe(sample_text, **gen_kwargs)[0]["summary_text"])

loading configuration file peg_model/config.json
Model config PegasusConfig {
  "_name_or_path": "peg_model",
  "activation_dropout": 0.1,
  "activation_function": "relu",
  "add_bias_logits": false,
  "add_final_layer_norm": true,
  "architectures": [
    "PegasusForConditionalGeneration"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 16,
  "decoder_start_token_id": 0,
  "dropout": 0.1,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 16,
  "eos_token_id": 1,
  "extra_pos_embeddings": 1,
  "forced_eos_token_id": 1,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "length_penalty": 0.8,
  "max_length": 128,
  "max_po


Reference Summary:
Sapna Gill arrested by Mumbai police for allegedly attacking Indian cricketer Prithvishaw with a baseball bat. Shameful viral video of Indian cricketer Prithvi Shaw being attacked by a woman and her friend should be punished severely.

Model Summary:
bhojpuri actress sapnagill arrested by police for allegedly attack indian cricketer prithvishaw prithvishaw .<n>Sampagill viral video on social medium whatever it be but it be not a human it be an animal it be a request to the modi govt that it should be punish severely immediate action .
